Podstawowe biblioteki do pracy z datami i czasem to `datetime` i `dateutil`. 

In [ ]:
from datetime import datetime
datetime(year=2015, month=7, day=4)

Korzystając z `dateutil` możemy np parsować daty podane jako tekst:

In [ ]:
from dateutil import parser
date = parser.parse('4th of July, 2015')

In [ ]:
date

A obiekty `datetime` zamieniać na string:

In [ ]:
date.strftime('%A')

https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

Ciekawą biblioteką jest również `pytz` które służy do pracy ze strefami czasowymi. 

Obiekty `datetime` są natomiast skomplikowane w użyciu na dużą skalę.

### numpy.datetime64

In [ ]:
import numpy as np

In [ ]:
date = np.array('2015-07-04', dtype=np.datetime64)

In [ ]:
date

Na takim typie danym możemy wykonywać operacje wektorowe:

In [ ]:
date + np.arange(12)

Jako, że pamięc przeznaczona na pojedynczy obiekt jest stała musimy być świadomi kompromisu pomiędzy dokładnością czasu, a rozmiarem zakresu dat jaki możemy przetworzyć.

In [ ]:
np.datetime64('2015-07-04')

In [ ]:
np.datetime64('2015-07-04 12:00')

In [ ]:
np.datetime64('2015-07-04 12:59:59.50', 'ns')

### Jak to wygląda w Pandasie?

Pandas stara się pogodzić oba światy dostarczając obiekt `Timestamp`

In [ ]:
import pandas as pd

date = pd.to_datetime('4th of July, 2015')

In [ ]:
date

In [ ]:
date.strftime('%A')

In [ ]:
date = date + pd.to_timedelta(np.arange(12), 'D')

In [ ]:
date

### Indeksowanie danych czasem

In [ ]:
index = pd.DatetimeIndex(['2014-07-04', '2014-08-04',
                          '2015-07-04', '2015-08-04'])
data = pd.Series([0, 1, 2, 3], index=index)
data

Możemy teraz filtrować dane korzystając z czasu:

In [ ]:
data['2014-07-04':'2015-07-04']

Lub przekazując np rok:

In [ ]:
data['2015']

Typy danych dotyczące czasu w Pandas:

- time stampy - pd.Timestamp
- okresy - pd.Period
- time deltas pd.Timedelta

In [ ]:
dates = pd.to_datetime([datetime(2015, 7, 3), '4th of July, 2015',
                       '2015-Jul-6', '07-07-2015', '20150708'])
dates

In [ ]:
dates.to_period('D')

In [ ]:
dates - dates[0]

### Ciągi dat

In [ ]:
pd.date_range('2015-07-03', '2015-07-10')

In [ ]:
pd.date_range('2015-07-03', '2015-07-10', freq='H')

### Resampling, shifting, windowing

In [ ]:
from pandas_datareader import data

goog = data.DataReader('GOOG',
                       start='2015',
                       end='2020',
                       data_source='stooq')

In [ ]:
goog.head()

In [ ]:
goog = goog['Close']

In [ ]:
goog

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()

In [ ]:
goog.plot();

Pracując z danymi czasowymi często chcemy je zsamplować po różnych okresach, pomaga tu metoda `resample` lub `asfreq`

In [ ]:
goog

In [ ]:
goog.resample('BA')

In [ ]:
goog.plot(alpha=0.5, style='-')
goog.resample('BA').mean().plot(style=':')
goog.asfreq('BA').plot(style='--')

plt.legend(['input', 'resample', 'asfreq'],
           loc='upper left');

- resample - zwraca średnią z poprzedniego okresu
- asfreq zwraca ostatnią wartość z okresu

For up-sampling, resample() and asfreq() are largely equivalent, though resample has many more options available. In this case, the default for both methods is to leave the up-sampled points empty, that is, filled with NA values. 

In [ ]:
fig, ax = plt.subplots(2, sharex=True)
data = goog.iloc[:10]

data.asfreq('D').plot(ax=ax[0], marker='o')

data.asfreq('D', method='bfill').plot(ax=ax[1], style='-o')
data.asfreq('D', method='ffill').plot(ax=ax[1], style='--o')
ax[1].legend(["back-fill", "forward-fill"]);

In [ ]:
fig, ax = plt.subplots(2, sharey=True)

# apply a frequency to the data
goog = goog.asfreq('D', method='pad')

goog.plot(ax=ax[0])
goog.shift(900).plot(ax=ax[1]) # przesuń dane o 900 dni

# legends and annotations
local_max = pd.to_datetime('2016-11-05')
offset = pd.Timedelta(900, 'D')

ax[0].legend(['input'], loc=2)
ax[0].get_xticklabels()[2].set(weight='heavy', color='red')
ax[0].axvline(local_max, alpha=0.3, color='red')

ax[1].legend(['shift(900)'], loc=2)
ax[1].get_xticklabels()[2].set(weight='heavy', color='red')
ax[1].axvline(local_max + offset, alpha=0.3, color='red')


Taka operacja przydaje się gdy chcemy policzyć różnicę w czasie. Np w ten sposób możemy policzyć roczny zwrot:

In [ ]:
(goog.shift(-365) / goog - 1) * 100

In [ ]:
ROI = 100 * (goog.shift(-365) / goog - 1)
ROI.plot()
plt.ylabel('% Return on Investment');

### Rolling windows

In [ ]:
rolling = goog.rolling(365, center=True)

In [ ]:
data = pd.DataFrame({'input':goog,
                     'rolling-mean': rolling.mean(),
                     'rolling-std': rolling.std()})

In [ ]:
data.plot(style=['-', '--', ':'])

In [ ]:
data = pd.read_csv('/Users/sapkowskis/Downloads/Fremont_Bridge_Bicycle_Counter.csv',
                   index_col='Date',
                   parse_dates=True)

In [ ]:
data.head()

In [ ]:
data.columns = ['Total', 'West', 'East']

In [ ]:
data.dropna().describe()

In [ ]:
data.plot()

In [ ]:
# too dense

In [ ]:
weekly = data.resample('W').sum()

In [ ]:
weekly.plot(style=[':', '--', '-'])

In [ ]:
daily = data.resample('D').sum()

In [ ]:
daily.rolling(30,
              center=True,
             ).sum().plot(style=[':', '--', '-'])

In [ ]:
data.groupby(data.index.time).mean().plot()

In [ ]:
by_weekday = data.groupby(data.index.dayofweek).mean()
by_weekday.index = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
by_weekday.plot(style=[':', '--', '-']);

In [ ]:
weekend = np.where(data.index.weekday < 5, 'Weekday', 'Weekend')
by_time = data.groupby([weekend, data.index.time]).mean()

In [ ]:
import matplotlib.pyplot as plt
hourly_ticks = 4 * 60 * 60 * np.arange(6)

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
by_time.loc['Weekday'].plot(ax=ax[0], title='Weekdays',
                           xticks=hourly_ticks, style=[':', '--', '-'])
by_time.loc['Weekend'].plot(ax=ax[1], title='Weekends',
                           xticks=hourly_ticks, style=[':', '--', '-']);

